In [ ]:
import numpy as np
import pandas as pd
from numba import njit, jit

from math import log, sqrt, pi, exp, erf, ceil, floor
from scipy.stats import norm, pearsonr
from scipy.stats.mstats import gmean

from matplotlib import pyplot as plt
from tqdm import tqdm, trange
from time import perf_counter
import gc

""" Plt font size """
plt.rcParams['font.size'] = '13'

Time per 10,000 samples = 3.09 sec

# Part 1

In [ ]:
""" Euler's Method """
def gbm_euler(S0, r, sigma, T, M, path_simulation=False):
    zm = norm.rvs(loc=0., scale=1, size=M)
    dt = T / M
    Sm = np.zeros((M+1, 2), dtype=float)   ;   Sm[0, 0] = S0
    for m in range(1, M+1):
        Sm[m, 0] = Sm[m-1, 0] + r*Sm[m-1, 0]*dt + sigma*Sm[m-1, 0]*sqrt(dt)*zm[m-1]
        Sm[m, 1] = m * dt
    if path_simulation:
        return Sm
    return Sm[-1,0]

In [ ]:
""" Geometric Brownian Motion - St Simulation"""
S0 = 100
sigma = .20
r = 0.06
T = 1.
M = 253
output = gbm_euler(S0, r, sigma, T, M, path_simulation=True)

temp_output = pd.DataFrame(output, columns=['St', 'Time'])
temp_output = temp_output.set_index('Time')
temp_output.plot(figsize=(20,4), grid=True, title="Simulated Stock Movement Using Geometric Brownian Motion", ylabel="Stock Price", label=f'Euler Method M={M}')

----------------------------------------------------------------------------------------------------------------------------------------------------------------
## Payoff Distribution and Convergence for Different Number of Paths (M)

In [ ]:
simulations = 100_000
payoff_dist = np.empty(simulations)
for inx in trange(simulations):
    K = 99 ; N = 254
    payoff_dist[inx] = max(K- gbm_euler(S0, r, sigma, T, N), 0)

print(f'Probability of tbe option expiring in the Money = {payoff_dist[payoff_dist > 0].shape[0] / simulations *100:.5f} %')
print(f'Expected Payoff = {payoff_dist.mean()*np.exp(-r*T):.5f} EUR \nStandard Deviation of E(Payoff) = {payoff_dist.std():.5f} EUR')
ax = pd.DataFrame(payoff_dist[payoff_dist>0], columns=['Payoff']).plot(kind='hist', bins=1000, density=True, figsize=(20,4), grid=True, title="Distribution of Positive Payoffs")
ax.set_xlabel('Payoff of Put Option (EUR)')

In [ ]:
""" Bootstrapping Technique """
def bootstrapping(M, K = 99, sigma = 0.2, SE_report=False):
    def _gbm_euler(S0, r, sigma, T, M, zm):
        dt = T / M
        Sm = np.zeros((M+1), dtype=float)   ;   Sm[0] = S0
        for m in range(1, M+1):
            Sm[m] = Sm[m-1] + r*Sm[m-1]*dt + sigma*Sm[m-1]*sqrt(dt)*zm[m-1]
        return Sm[-1]

    S0 = 100
    # sigma = .20
    r = 0.06
    T = 1.
    N = 254

    # K = 99

    M = np.sqrt(M).astype(int)   ;   bootstrap_trials = M

    _results = - np.ones((M, bootstrap_trials))
    for b_trial in range(bootstrap_trials):
        zm = np.random.normal(loc=0., scale=1, size=(N, ceil(M/2)))
        zm = np.concatenate((zm, -zm[:,:floor(M/2)]), axis=1)  ;  """ Antithetic Variable """
        for _inx in range(M):
            _results[_inx, b_trial] = max(K- _gbm_euler(S0, r, sigma, T, N, zm[:, _inx]), 0) * np.exp(-r*T)

    if SE_report:
        return np.mean(_results, axis=0).std() / np.sqrt(_results.shape[1])
    return np.mean(_results, axis=0).mean(), np.mean(_results, axis=0).std() / np.sqrt(_results.shape[1])

In [ ]:
start_M = 10    ;   end_M = 1_000_000 ;   points_in_graph = 30

M_values = np.logspace(start=np.log10(start_M), stop=np.log10(end_M), num=points_in_graph, base=10)

""" Creates a DataFrame to fill with values """
results_convergence_df = pd.DataFrame(np.zeros((points_in_graph, 2)), index=M_values , columns=['Expected Payoff - Put Option', 'Standard Error'])
results_convergence_df.index.name = 'Number of Paths (M)'

for M in tqdm(M_values):
    results_convergence_df.loc[M, :] = bootstrapping(M) # returns: mean, SE

results_convergence_df

In [ ]:
BS_analytical_solution = 4.778969051891707
print(f'Expected Payoff = {(results_convergence_df.iloc[-1, 0] - BS_analytical_solution)/BS_analytical_solution * 100:.5f} %')
print(f'Percentage Difference from Analytical Solution = {(results_convergence_df.iloc[-1, 0] - BS_analytical_solution)/BS_analytical_solution * 100:.5f} %')

In [ ]:
axs = results_convergence_df.plot(figsize=(20,8), grid=True, title='Convergence of Mean Payoff with number of Paths Simulated', subplots=True, logx=True)
axs[0].set_ylabel('Payoff (EUR)')
axs[0].axhline(4.778969051891707, color='r', linestyle='-', alpha=.5, label='Analytical Solution using BS') ;   axs[0].legend()

axs[1].plot(M_values, [ 6/np.sqrt(_m_value)  for _m_value in M_values], label=r'$f(M) = c/\sqrt{M}, c \approx 6$')    ;   axs[1].legend()

axs[1].set_yscale('log')
axs[1].set_ylabel('Standard Error (SE)')

## Different MC Implementation Approaches
> Vanilla MC - 100,000 samples
Mean = 4.757270
Standard Error = 0.025220

> Bootstrapping Results with and without Antithetic (AT) VR - with 100,000 samples and sqrt(samples) df
------ Without AT ------
Mean = 4.7845864899592705
Standard Error = 0.026560446487301444
------ With AT ------
Mean = 4.764542084945663
Standard Error = 0.020000435977867624

----------------------------------------------------------------------------------------------------------------------------------------------------------------
## Strike Price Graph

In [ ]:
M = 100_000 ;   size_of_graph = 40 ;   start_x = 1  ;   end_x = 200

K_values = np.linspace(start_x, end_x, size_of_graph, dtype=int)

""" Creates a DataFrame to fill with values """
results_k_df = pd.DataFrame(np.zeros((size_of_graph, 3)), index=K_values , columns=['Expected Payoff - Put Option', 'Standard Error (SE)', 'Relative Standard Error (RSE)'])
results_k_df.index.name = 'Strike Price (EUR)'

for K in tqdm(K_values):
    _m, _se = bootstrapping(M, K=K)
    results_k_df.loc[K] = (_m, _se, _se/_m *100.) if _m!= 0 else np.nan

results_k_df

In [ ]:
axs = results_k_df.plot(figsize=(20,10), grid=True, title='Standard Error and Relative Error for Different Strike Prices', subplots=True)
axs[0].set_ylabel('Payoff (EUR)')   ;   axs[1].set_ylabel('Standard Error')   ;   axs[2].set_ylabel('RSE')
axs[2].set_yscale('log')

axs[0].axvline(100, color='r', linestyle='-', alpha=.5, label=r'$S_0$')
axs[0].axvline(100*np.exp(.06), color='g', linestyle='-', alpha=.5, label=r'$S_0*e^{rT}$') ;   axs[0].legend()
axs[1].axvline(100, color='r', linestyle='-', alpha=.5, label=r'$S_0$')
axs[1].axvline(100*np.exp(.06), color='g', linestyle='-', alpha=.5, label=r'$S_0*e^{rT}$') ;   axs[1].legend()
axs[2].axvline(100, color='r', linestyle='-', alpha=.5, label=r'$S_0$')
axs[2].axvline(100*np.exp(.06), color='g', linestyle='-', alpha=.5, label=r'$S_0*e^{rT}$') ;   axs[2].legend()

plt.legend()

----------------------------------------------------------------------------------------------------------------------------------------------------------------
## Volatility Graph

In [ ]:
M = 100_000 ;   size_of_graph = 40 ;   start_v = .01  ;   end_v = 10

V_values = np.logspace(np.log10(start_v), np.log10(end_v), size_of_graph)

""" Creates a DataFrame to fill with values """
results_V_df = pd.DataFrame(np.zeros((size_of_graph, 3)), index=np.log10(V_values) , columns=['Expected Payoff - Put Option', 'Standard Error (SE)', 'Relative Standard Error (RSE)'])
results_V_df.index.name = 'Realized Volatility (σ)'

for V in tqdm(V_values):
    _m, _se = bootstrapping(M, sigma=V)
    results_V_df.loc[np.log10(V)] = (_m, _se, _se/_m *100) #  if _m > 0 else 0

results_V_df

In [ ]:
axs = results_V_df.plot(figsize=(20,10), grid=True, title='Standard Error and Relative Error for Different Volatility Levels', subplots=True) #, xticks=V_values)
axs[0].set_ylabel('Payoff (EUR)')   ;   axs[1].set_ylabel('Standard Error')   ;   axs[2].set_ylabel('RSE (%)')
axs[2].set_yscale('log')
axs[0].axhline(99, color='r', linestyle='-', alpha=.5, label='Strike Value - 99 EUR') ;   axs[0].legend()

plt.xticks(plt.xticks()[0][1:-1], [f'{10**x:.2f}' for x in plt.xticks()[0][1:-1]])

## 3D Plot - Strike Price / Volatility

In [ ]:
import multiprocessing
from joblib import Parallel, delayed

""" Simulation Parameters """
M = 100_000    ;    total_points = 900  ;    print(f'Total Paths Calculated (M) = {M*total_points}')# points should be [2500, 900]

start_sigma = 0.01  ;   end_sigma = 20
start_K = 1  ;   end_K = 200

""" Prepare Input """
size_of_graph = np.sqrt(total_points).astype(int)   # points should be [2500, 900]
V_values = np.logspace(np.log10(start_sigma), np.log10(end_sigma), size_of_graph, base=10)
# V_values = np.linspace(start_sigma, end_sigma, size_of_graph)
K_values = np.linspace(start_K, end_K, size_of_graph, dtype=int)

input_matrix = np.empty((0, 2))
for sigma in V_values:
    for K in K_values:
        input_matrix = np.append(input_matrix, np.array([[sigma, K]]), axis=0)

inputs = tqdm(input_matrix)


""" Define function that calculates """
def simulate(input_matrix):
    """ X """
    # x = np.log10(input_matrix[0])
    """ Y """
    # y = input_matrix[1]
    """ Z """
    _m, _se = bootstrapping(M, sigma=input_matrix[0], K=input_matrix[1])
    _r = (_se  /_m *100) if _m!= 0 else 0
    return np.array([np.log10(input_matrix[0]), input_matrix[1], _r])


""" Run Parallel Simulation """
num_cores = multiprocessing.cpu_count()
temp_output = Parallel(n_jobs=num_cores)(delayed(simulate)(row) for row in inputs)
""" Revert back to matrix form"""
# print(temp_output)
result_3D = np.empty((0, 3))
for array in temp_output:
    result_3D = np.append(result_3D, [array], axis=0)

In [ ]:
import matplotlib.ticker as mticker
from matplotlib import cm
fig = plt.figure(figsize=(15,12))
ax = plt.axes(projection='3d')
ax.plot_trisurf(result_3D[:, 0], result_3D[:, 1], np.log10(result_3D[:, 2]), cmap=cm.jet, linewidth=0.2)

def log_tick_formatter(val, pos=None):
    return r"$10^{:.0f}$".format(val)

ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))

plt.xticks(plt.xticks()[0][1:-1], [f'{10**x:.2f}' for x in plt.xticks()[0][1:-1]])

ax.set_ylabel('Strike Price (EUR)')   ;   ax.set_xlabel('Volatility')   ;   ax.set_zlabel('RSE (%)')
ax.set_title("RSE for Different Strike Price and Volatility Levels")

_________________________________________________________________________________________________________________________________________________
# Part 2
## Question 1 - MC for European Put

In [ ]:
""" Analytical Result - Black Scholes """
def bs_call(S, K, T, sigma, r):
    d1 = (log(S/K) + (r + sigma**2/2.) * T) / (sigma*sqrt(T))
    d2 = d1 - sigma * sqrt(T)

    # V = S*norm.cdf(d1) -K*exp(-r*T)*norm.cdf(d2)
    # D = norm.cdf(d1)
    D = (1.0 + erf(d1 / sqrt(2.0))) / 2.0
    V = S*D  -K*exp(-r*T)* (1.0 + erf(d2 / sqrt(2.0))) / 2.0

    return V, D

def bs_put(S, K, T, sigma, r):
    d1 = (log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)

    """ negative in front - negative in front of d1 """
    D = - (1.0 + erf(-d1 / sqrt(2.0))) / 2.0
    V = S*D + K*exp(-r*T)* (1.0 + erf(-d2 / sqrt(2.0))) / 2.0

    return V, D

In [ ]:
""" Monte Carlo Method of Approximating the Value of a Call Option """
def call_option_MC(S0=100, K=99, T=1., sigma=.2, r=.06, M=10_000, seed=None):
    def _gbm_euler(S0, r, sigma, T, M, zm):
        dt = T / M
        Sm = np.zeros((M+1), dtype=float)   ;   Sm[0] = S0
        for m in range(1, M+1):
            Sm[m] = Sm[m-1] + r*Sm[m-1]*dt + sigma*Sm[m-1]*sqrt(dt)*zm[m-1]
        return Sm[-1]

    epsilon = 1
    N = 252 # trading days in a year
    M = np.sqrt(M).astype(int)

    """ Create the IID Random Normal Variables : For GBM Simulation """
    np.random.seed(seed=seed)
    zm1 = np.random.normal(loc=0., scale=1, size=(N, M, M))
    np.random.seed(seed=seed)
    zm2 = np.random.normal(loc=0., scale=1, size=(N, M, M))
    _results = - np.ones((M, M))    ;   _results_2 = - np.ones((M, M))
    for _M1 in range(M):
        for _M2 in range(M):
            _results[_M1, _M2] = max(K- _gbm_euler(S0, r, sigma, T, N, zm1[:, _M1, _M2]), 0) * np.exp(-r*T)
            """ Bump and Revalue Method of Calculating Delta """
            _results_2[_M1, _M2] = max(K- _gbm_euler(S0+epsilon, r, sigma, T, N, zm2[:, _M1, _M2]), 0) * np.exp(-r*T)

    Vs = _results    ;   Vs_2 = _results_2
    Ds = (Vs_2 - Vs) / epsilon

    SE_V = np.mean(Vs, axis=0).std() / np.sqrt(_results.shape[1])
    SE_D = np.mean(Ds, axis=0).std() / np.sqrt(Ds.shape[1])
    return Vs.mean(), SE_V, Ds.mean(), SE_D

# 10_000  = -0.3171157567818145 , e=1
# 100_000 = -0.35178048916918736 , e=1
# 10_000  = -0.32440869426635943 , e=.1
# 100_000 = -0.32476038600091583, , e=.1
# call_option_MC(S0=S0, K=K, T=T, sigma=sigma, r=r ,M=100_000, seed=21)

### Results: Table --> Payoff and Delta for Vanilla MC w/o and w/ seed

In [ ]:
M_values = 10**np.array([2, 3, 4, 5])  # number of paths to simulate
_column_names = ['E[Payoff of Put Option]', 'Standard Error', 'Payoff Difference with BS', 'Delta', 'Delta Difference with BS Delta']
S0, K, T, sigma, r = (100, 99, 1., .2, .06)

""" Get Analytical Results """
V_bs, D_bs = bs_put(S0, K, T, sigma, r)    ;   print(f'>>> Analytical Solution (V,D) = {V_bs:.5f} , {D_bs:.5f}')

""" Calculate the Effect of M and using and not using a seed """
results_no_seed_df = pd.DataFrame(np.empty((M_values.shape[0],5)), index=M_values, columns=_column_names)
results_with_seed_df = pd.DataFrame(np.empty((M_values.shape[0],5)), index=M_values, columns=_column_names)
for _M in tqdm(M_values):
    _r_without = call_option_MC(S0=S0, K=K, T=T, sigma=sigma, r=r ,M=_M, seed=None)
    results_no_seed_df.loc[_M] = [_r_without[0], _r_without[1], _r_without[0]-V_bs, _r_without[2], _r_without[2]-D_bs]

    _r_with = call_option_MC(S0=S0, K=K, T=T, sigma=sigma, r=r ,M=_M, seed=21)
    results_with_seed_df.loc[_M] = [_r_with[0], _r_with[1], _r_with[0]-V_bs, _r_with[2], _r_with[2]-D_bs]

In [ ]:
print('>>> Results when using random seed for the creation of the IDD Random Variables for the GBM')
results_no_seed_df

In [ ]:
print('>>> Results when using predefined seed for the creation of the IDD Random Variables for the GBM')
results_with_seed_df

In [ ]:
S0, K, T, sigma, r = (100, 99, 1., .2, .06)
V_bs, D_bs = bs_put(S0, K, T, sigma, r)

_M_array = 5*np.logspace(1, 4, 20, base=10, dtype=int)
results_num_steps = pd.DataFrame(np.empty((_M_array.shape[0],2)), index=_M_array, columns=['Payoff Difference with BS', 'Delta Difference with BS Delta'])
for _M in tqdm(_M_array):
    _r = call_option_MC(S0=S0, K=K, T=T, sigma=sigma, M=_M, seed=None)
    results_num_steps.loc[_M] = [_r[0]-V_bs, _r[2]-D_bs]

results_num_steps.plot(figsize=(20,6), xlabel="N (Steps)", ylabel='Difference with Black-Scholes', title="Accuracy of MC Approximation", grid=True, logx=True, subplots=True)
results_num_steps

In [ ]:
________________________________________________________________________________________________________________________________________________________________________
## Question 2 - Digital Option

In [ ]:
from numpy import log as ln
""" Analytical Solution Digital Option """
def digital_call_valuation_analytical(S, K, T, sigma, r):
    Q = 1  # Amount to be delivered at Expiration date if in the money

    d2 = (ln(S/K) + (r - 0.5 * sigma**2) * T) / (sigma * sqrt(T))

    V = Q * np.exp(-r*T) * norm.cdf(d2)
    D = (np.exp(-r*T)*norm.pdf(d2)) / (S*sigma*np.sqrt(T))
    return V, D

def digital_put_valuation_analytical(S, K, T, sigma, r):
    Q = 1  # Amount to be delivered at Expiration date if in the money

    d2 = (ln(S/K) + (r - 0.5 * sigma**2) * T) / (sigma * sqrt(T))

    V = Q * np.exp(-r*T) * norm.cdf(-d2)
    D = -(np.exp(-r*T)*norm.pdf(-d2)) / (S*sigma*np.sqrt(T))
    return V, D

print('Digital Call (V, D):', digital_call_valuation_analytical(100, 99, 1., .2, .06))
print('Digital Put (V, D):', digital_put_valuation_analytical(100, 99, 1., .2, .06))

In [ ]:
""" Simple - MC Method for Approximating the Value of a Digital Call Option """
def digital_call_option_MC(S0=100, K=99, T=1., sigma=.2, r=.06, M=10_000, seed=None):
    def _gbm_euler(S0, r, sigma, T, M, zm):
        dt = T / M
        Sm = np.zeros((M+1), dtype=float)   ;   Sm[0] = S0
        for m in range(1, M+1):
            Sm[m] = Sm[m-1] + r*Sm[m-1]*dt + sigma*Sm[m-1]*sqrt(dt)*zm[m-1]
        return Sm[-1]

    epsilon = 0.1
    N = 252 # trading days in a year
    M = np.sqrt(M).astype(int)

    """ Create the IID Random Normal Variables : For GBM Simulation """
    np.random.seed(seed=seed)
    zm = np.random.normal(loc=0., scale=1, size=(N, M, M))

    _results = - np.ones((M, M))
    _results_2 = - np.ones((M, M))
    for b_trial in range(M):
        for _inx in range(M):
            x1 = _gbm_euler(S0, r, sigma, T, N, zm[:, _inx, b_trial]) - K
            """ Bump and Revalue Method of Calculating Delta """
            x2 = _gbm_euler(S0+epsilon, r, sigma, T, N, zm[:, _inx, b_trial]) - K
            _results[_inx, b_trial] = 1 if x1 > 0 else 0
            _results_2[_inx, b_trial] = 1 if x2 > 0 else 0
    _option_value = np.exp(-r*T)*_results.mean()    ;   _option_value_2 = np.exp(-r*T)*_results_2.mean()

    delta = (_option_value_2 - _option_value) / epsilon

    SE = np.mean(_results, axis=0).std() / np.sqrt(_results.shape[1])
    return _option_value, SE, delta

### Vanilla MC Results

In [ ]:
M_values = 10**np.array([2, 3, 4, 5])  # number of paths to simulate
_column_names = ['E[Payoff of Digital Call Option]', 'Standard Error', 'Payoff Difference with BS', 'Delta', 'Delta Difference with BS Delta']
S0, K, T, sigma, r = (100, 99, 1., .2, .06)

""" Get Analytical Results """
V_bs_digital, D_bs_digital = digital_call_valuation_analytical(S0, K, T, sigma, r)    ;   print(f'>>> Analytical Solution (V,D) = {V_bs_digital:.5f} , {D_bs_digital:.5f}')

""" Calculate the Effect of M and using and not using a seed """
results_B = pd.DataFrame(np.empty((M_values.shape[0],5)), index=M_values, columns=_column_names)
for M in tqdm(M_values):
    _r_without = digital_call_option_MC(S0=S0, K=K, T=T, sigma=sigma, r=r, M=M, seed=None)
    results_B.loc[M] = [_r_without[0], _r_without[1], _r_without[0]-V_bs_digital, _r_without[2], _r_without[2]-D_bs_digital]

In [ ]:
results_B

### Pathwise Method of approximating Delta

In [ ]:
""" Pathwise - MC Method for Approximating the Value of a Digital Call Option """
def digital_call_option_MC_PW(S0=100, K=99, T=1., sigma=.2, r=.06, M=10_000, seed=None, _coef=0):
    if not _coef: _coef = 8
    def _gbm_exact(S0, r, sigma, T, M, zm):
        dt = T / M
        Sm = np.zeros((M+1), dtype=float)   ;   Sm[0] = S0
        for m in range(1, M+1):
            Sm[m] = Sm[m-1] + r*Sm[m-1]*dt + sigma*Sm[m-1]*sqrt(dt)*zm[m-1]
        return Sm[-1]

    _coefficient_limit = 700/(10*_coef) # to avoid overflow error in the exponential function inside the smoothing function
    N = 252 # trading days in a year
    M = np.sqrt(M).astype(int)

    """ Create the IID Random Normal Variables : For GBM Simulation """
    np.random.seed(seed=seed)
    zm = np.random.normal(loc=0., scale=1, size=(N, M, M))

    _results = - np.empty((M, M, 2))
    for _M1 in range(M):
        for _M2 in range(M):
            _St = _gbm_exact(S0, r, sigma, T, N, zm[:, _M1, _M2])   ;   _payoff = _St - K
            if -_coefficient_limit < _payoff < _coefficient_limit:
                _results[_M1, _M2, 0] = 1 / (1 + np.exp(-_coef *_payoff))
                _results[_M1, _M2, 1] = (_coef * np.exp(_coef * _payoff)) / (np.exp(_coef *_payoff) + 1)**2 # Path Wise Approximation of Delta
            else:
                _results[_M1, _M2, 0] = 1 if _payoff > 0 else 0
                _results[_M1, _M2, 1] = 0

    _option_value = np.exp(-r*T) * _results[:,:,0].mean()  #
    delta = np.exp(-r*T) * _results[:,:,1].mean()

    SE_V = np.mean(_results[:, :, 0], axis=0).std() / np.sqrt(_results.shape[1])
    SE_D = np.mean(_results[:, :, 1], axis=0).std() / np.sqrt(_results.shape[1])
    return _option_value, SE_V, delta, SE_D

In [ ]:
""" Plot Sigmoid """
x = np.linspace(-5, 5, num=100)
y1 = [1 / (1+np.exp(-x)) for x in x]
y2 = [1 / (1+np.exp(-2*x)) for x in x]
y5 = [1 / (1+np.exp(-5*x)) for x in x]
y10 = [1 / (1+np.exp(-10*x)) for x in x]
_dict = {"c = 1" : y1, "c = 2" : y2, "c = 5" : y5, "c = 10" : y10}

pd.DataFrame(_dict, index=x).plot(figsize=(15,4), title="Sigmoid Function used to Smooth a Digital's Option Payoff", xlabel=r"Stock Price at T minus Strike price ($S_T - K$)", ylabel="Option Payoff", grid=True)

### Tuning the slope of the binary option - sigmoid "coefficient"

In [ ]:
digital_call_analytical = digital_call_valuation_analytical(100, 99, 1., .2, .06)

size = 50
results_sigmoid_coefficient = np.empty((size, 3))
for _index, _c in tqdm(enumerate(np.linspace(-4, 5, num=size)), total=size):
    _r = digital_call_option_MC_PW(seed=None, _coef=10**_c, M=10_000)
    results_sigmoid_coefficient[_index, 0] = 10**_c
    results_sigmoid_coefficient[_index, 1] = _r[0] - digital_call_analytical[0]
    results_sigmoid_coefficient[_index, 2] = _r[2] - digital_call_analytical[1]

axs = pd.DataFrame(results_sigmoid_coefficient[:, 1:], index=results_sigmoid_coefficient[:, 0], columns=['Difference with Analytical Value', 'Difference with Analytical Delta']).plot(figsize=(20,6), xlabel="Sigmoid Coefficient", ylabel='Difference with BS', title="Optimizing Sigmoid Slope for Binary Options - Pathwise", grid=True, logx=True, subplots=True)
axs[0].axhline(0, color='r', linestyle='-', alpha=.5)   ;   axs[1].axhline(0, color='r', linestyle='-', alpha=.5)
results_sigmoid_coefficient

### Likelihood Ratio

In [ ]:
""" Pathwise - MC Method for Approximating the Value of a Digital Call Option """
def digital_call_option_MC_LH(S0=100, K=99, T=1., sigma=.2, r=.06, M=10_000, seed=None):
    def _gbm_exact(S0, r, sigma, T, M, zm):
        dt = T / M
        Sm = np.zeros((M+1), dtype=float)   ;   Sm[0] = S0
        for m in range(1, M+1):
            Sm[m] = Sm[m-1] + r*Sm[m-1]*dt + sigma*Sm[m-1]*sqrt(dt)*zm[m-1]
        return Sm[-1]

    N = 252 # trading days in a year
    M = np.sqrt(M).astype(int)

    """ Create the IID Random Normal Variables : For GBM Simulation """
    np.random.seed(seed=seed)
    zm = np.random.normal(loc=0., scale=1, size=(N, M, M))

    _results = - np.empty((M, M, 2))
    for _M1 in range(M):
        for _M2 in range(M):
            _St = _gbm_exact(S0, r, sigma, T, N, zm[:, _M1, _M2])
            _payoff = 1 if _St > K else 0
            _results[_M1, _M2, 0] = _payoff
            _results[_M1, _M2, 1] = _payoff * (np.log( _St / S0 ) - (r - 0.5 * sigma**2) * T) / (S0 * sigma**2 * T)

    _option_value = np.exp(-r*T) * _results[:,:,0].mean()
    delta = np.exp(-r*T) * _results[:,:,1].mean()

    SE_V = np.mean(_results[:, :, 0], axis=0).std() / np.sqrt(_results.shape[1])
    SE_D = np.mean(_results[:, :, 1], axis=0).std() / np.sqrt(_results.shape[1])
    return _option_value, SE_V, delta, SE_D

### Results - Pathwise & Likelihood

In [ ]:
""" Results -> Table """
M_values = 10**np.array([2, 3, 4, 5])  # number of paths to simulate
_column_names_V = ['E[Payoff]PW', 'Standard Error', 'Payoff Difference with BS', 'E[Payoff]LH', 'Standard Error', 'Payoff Difference with BS', 'Ex Time PW', 'Ex Time LH']
_column_names_D = ['E[Delta]PW', 'Standard Error', 'Delta Difference with BS', 'E[Delta]LH', 'Standard Error', 'Delta Difference with BS', 'Ex Time PW', 'Ex Time LH']
S0, K, T, sigma, r = (100, 99, 1., .2, .06)

""" Get Analytical Results """
V_bs_digital, D_bs_digital = digital_call_valuation_analytical(S0, K, T, sigma, r)    ;   print(f'>>> Analytical Solution (V,D) = {V_bs_digital:.5f} , {D_bs_digital:.5f}')

""" Calculate the Effect of M and using and not using a seed """
V_analysis = pd.DataFrame(np.empty((M_values.shape[0],8)), index=M_values, columns=_column_names_V)
D_analysis = pd.DataFrame(np.empty((M_values.shape[0],8)), index=M_values, columns=_column_names_D)
for M in tqdm(M_values):
    _st1 = perf_counter()
    _r_pw = digital_call_option_MC_PW(S0=S0, K=K, T=T, sigma=sigma, r=r, M=M, seed=21)    ;   _et1 = perf_counter() - _st1

    _st2 = perf_counter()
    _r_lh = digital_call_option_MC_LH(S0=S0, K=K, T=T, sigma=sigma, r=r, M=M, seed=21)    ;   _et2 = perf_counter() - _st2

    V_analysis.loc[M] = [_r_pw[0], _r_pw[1], _r_pw[0]-V_bs_digital,  _r_lh[0], _r_lh[1], _r_lh[0]-V_bs_digital, _et1, _et2]
    D_analysis.loc[M] = [_r_pw[2], _r_pw[3], _r_pw[2]-D_bs_digital,  _r_lh[2], _r_lh[3], _r_lh[2]-D_bs_digital, _et1, _et2]

In [ ]:
V_analysis

In [ ]:
D_analysis

In [ ]:
""" Results -> Graph """
M_values = np.logspace(2, 4.5, num=30, base=10)

V_analysis = pd.DataFrame(np.empty((M_values.shape[0],3)), index=M_values, columns=['Vanilla MC', 'Pathwise', 'Likelihood Ratio'])
D_analysis = pd.DataFrame(np.empty((M_values.shape[0],3)), index=M_values, columns=['Vanilla MC', 'Pathwise', 'Likelihood Ratio'])
for _M in tqdm(M_values):
    _r_va = digital_call_option_MC(S0=S0, K=K, T=T, sigma=sigma, r=r, M=_M, seed=1)
    _r_pw = digital_call_option_MC_PW(S0=S0, K=K, T=T, sigma=sigma, r=r, M=_M, seed=1)
    _r_lh = digital_call_option_MC_LH(S0=S0, K=K, T=T, sigma=sigma, r=r, M=_M, seed=1)

    V_analysis.loc[_M] = [_r_va[0]-V_bs_digital, _r_pw[0]-V_bs_digital,  _r_lh[0]-V_bs_digital]
    D_analysis.loc[_M] = [_r_va[2]-D_bs_digital, _r_pw[2]-D_bs_digital,  _r_lh[2]-D_bs_digital]

In [ ]:
ax = V_analysis.plot(figsize=(20,4), logx=True, grid=True, title="Monte Carlo Vanilla vs Pathwise vs Likelihood Ratio Approach for Calculating Payoff", xlabel='Number of Paths Simulated (M)', ylabel='Difference with Analytical Solution')
ax.axhline(0, color='r', linestyle='-', alpha=.5)   ;   axs[1].axhline(0, color='r', linestyle='-', alpha=.5)
# V_analysis

In [ ]:
ax = D_analysis.plot(figsize=(20,4), logx=True, grid=True, title="Monte Carlo Vanilla vs Pathwise vs Likelihood Ratio Approach for Calculating Greeks", xlabel='Number of Paths Simulated (M)', ylabel='Difference with Analytical Solution', logy=False)
ax.axhline(0, color='r', linestyle='-', alpha=.5)   ;   axs[1].axhline(0, color='r', linestyle='-', alpha=.5)

--------------------------------------------------------------------------------------------------------------------------------------------
# Part 3: Asian Options and Control Variets

""" Euler's Method """
def gbm_euler(S0, r, sigma, T, N):
    zm = norm.rvs(loc=0., scale=1, size=N)
    dt = T / N
    Sm = np.zeros(N+1, dtype=float)   ;   Sm[0] = S0
    for m in range(1, N+1):
        Sm[m] = Sm[m-1] + r*Sm[m-1]*dt + sigma*Sm[m-1]*sqrt(dt)*zm[m-1]
    return Sm

In [ ]:
""" Euler's Method """
def gbm_euler(S0, r, sigma, T, N):
    zm = norm.rvs(loc=0., scale=1, size=N)
    dt = T / N
    Sm = np.zeros(N+1, dtype=float)   ;   Sm[0] = S0
    for m in range(1, N+1):
        Sm[m] = Sm[m-1] + r*Sm[m-1]*dt + sigma*Sm[m-1]*sqrt(dt)*zm[m-1]
    return Sm

In [ ]:
def geometric_analytical(S0, r, sigma, T, N, M, K):
    # Define the sigma bar and r bar
    sigma_bar = sigma * sqrt((2 * N + 1)/(6 * (N + 1)))
    r_bar = 0.5 * (r - 0.5 * sigma **2 + sigma_bar **2)

    # Define d1 and d2
    d1 = (log(S0/K) + (r_bar + 1/2 * sigma_bar ** 2) * T)/(sigma_bar * sqrt(T))
    d2 = d1 - (sigma_bar * sqrt(T))

    # Find the analytical expression
    analytical_value = exp(-r * T) * (S0 * exp(r_bar * T) * norm.cdf(d1) - K * norm.cdf(d2))

    return analytical_value

In [ ]:
# Geometric Monte Carlo
def geometric_MC(S0, r, sigma, T, N, M, K):
    mc_values_geo = np.zeros(M)
    for m in range(M):
        Sm = gbm_euler(S0, r, sigma, T, N)
        
        # Get geometric average
        AN = gmean(Sm)

        mc_values_geo[m] = max(0, AN - K)

    return mc_values_geo

In [ ]:
def geometric_analytical(S0, r, sigma, T, N, M, K):
    # Define the sigma bar and r bar
    sigma_bar = sigma * sqrt((2 * N + 1)/(6 * (N + 1)))
    r_bar = 0.5 * (r - 0.5 * sigma **2 + sigma_bar **2)

    # Define d1 and d2
    d1 = (log(S0/K) + (r_bar + 1/2 * sigma_bar ** 2) * T)/(sigma_bar * sqrt(T))
    d2 = d1 - (sigma_bar * sqrt(T))

    # Find the analytical expression
    analytical_value = exp(-r * T) * (S0 * exp(r_bar * T) * norm.cdf(d1) - K * norm.cdf(d2))

    return analytical_value

# Arithmetic Monte Carlo
def aritmetic_MC(S0, r, sigma, T, N, M, K):
    mc_values_arit = np.zeros(M)
    mc_values_geo = np.zeros(M)
    for m in range(M):
        Sm = gbm_euler(S0, r, sigma, T, N)
        
        # Get arithmetic average
        AN = Sm.mean()
        
        mc_values_arit[m] = max(0, AN - K)
        
        # Get geometric average
        AN = gmean(Sm)

        mc_values_geo[m] = max(0, AN - K)

    return mc_values_arit, mc_values_geo

def control_variate_est(S0, r, sigma, T, N, M, K):
    analytical_value = geometric_analytical(S0, r, sigma, T, N, M, K)
    mc_values_arit, mc_values_geo = MC(S0, r, sigma, T, N, M, K)
    
    A_arit = mc_values_arit.mean()
    A_arit_sd = mc_values_arit.std()
    A_geo = mc_values_geo.mean()
    A_geo_sd = mc_values_geo.std()

    # Control variate estimate Asian option
    rho, _ = pearsonr(mc_values_geo, mc_values_arit)
    
    if A_geo_sd == 0:
        beta = 0
    else:
        beta = (A_arit_sd / A_geo_sd) * rho

    estimate = A_arit - beta * (A_geo - analytical_value)

    return estimate, A_arit

In [ ]:
def control_variate_est(S0, r, sigma, T, N, M, K):
    analytical_value = geometric_analytical(S0, r, sigma, T, N, M, K)
    #mc_values_geo = geometric_MC(S0, r, sigma, T, N, M, K)
    mc_values_arit, mc_values_geo = aritmetic_MC(S0, r, sigma, T, N, M, K)
    
    A_arit = mc_values_arit.mean()
    A_arit_sd = mc_values_arit.std()
    A_geo = mc_values_geo.mean()
    A_geo_sd = mc_values_geo.std()

    # Control variate estimate Asian option
#     rho = (log(A_arit) * log(A_geo)) / (2 * A_arit_sd * A_geo_sd)
    rho, _ = pearsonr(mc_values_geo, mc_values_arit)
    
    if A_geo_sd == 0:
        beta = 0
    elif A_arit_sd == 0:
        beta = rho
    else:
        beta = (A_arit_sd / A_geo_sd) * rho

    estimate = A_arit - beta * (A_geo - analytical_value)

    return estimate, A_arit

In [ ]:
# Input parameter values
S0 = 100
T = 1.
N = 50
K = 99
r = 0.06
sigma = 0.2
M = 100_000

# Experiment 1: compare the control variate estimate with the MC estimate without correcting
control_var_est, MC_arit_est = control_variate_est(S0, r, sigma, T, N, M, K)

print("Control Variate estimate:", control_var_est)
print("MC arithmetic estimate:", MC_arit_est)
print("Difference of control variate and arithmetic MC estimates (in %):", (control_var_est - MC_arit_est)/MC_arit_est)

## Changing M

In [ ]:
S0 = 100
T = 1.
N = 50
K = 99
r = 0.06
sigma = 0.2

num_samples = 100
#M_values = np.linspace(100, 100_000, num=num_samples, dtype=int)
M_values = np.logspace(start=1, stop=6, num=num_samples, base=10, dtype=int)
print(M_values)

M_estimates_c = np.zeros(num_samples)
M_estimates_MC = np.zeros(num_samples)
for i in tqdm(range(num_samples)):
    M = M_values[i]
    M_estimates_c[i], M_estimates_MC[i] = control_variate_est(S0, r, sigma, T, N, M, K)

In [ ]:
print(M_estimates_c)
print(M_estimates_MC)

In [ ]:
plt.figure(figsize=(10,6), dpi=300)
plt.plot(M_values, M_estimates_MC, label='MC Arithmetic Estimate')
plt.plot(M_values, M_estimates_c, label='Control Variate Estimate')
plt.xscale('log',base=10) 
plt.xlabel("Number of Paths Generated (M)")
plt.ylabel("Call Option Price Approximation (EUR)")
plt.title("Performance of Control Variates For Increasing Number of Path Generated")
plt.legend()
plt.grid()
plt.show()

## Changing K

In [ ]:
S0 = 100
T = 1.
N = 50
r = 0.06
sigma = 0.2
M = 100_000

num_samples = 100
K_values = np.linspace(1, 200, num=num_samples, dtype=int)
print(K_values)

K_estimates_c = np.zeros(num_samples)
K_estimates_MC = np.zeros(num_samples)
for i in tqdm(range(num_samples)):
    K = K_values[i]
    K_estimates_c[i], K_estimates_MC[i] = control_variate_est(S0, r, sigma, T, N, M, K)

In [ ]:
S0 = 100
T = 1.
N = 50
r = 0.06
sigma = 0.2
M = 1000

num_samples = 100
K_values = np.linspace(1, 200, num=num_samples, dtype=int)
print(K_values)

K_estimates_c_1000 = np.zeros(num_samples)
K_estimates_MC_1000 = np.zeros(num_samples)
for i in tqdm(range(num_samples)):
    K = K_values[i]
    K_estimates_c_1000[i], K_estimates_MC_1000[i] = control_variate_est(S0, r, sigma, T, N, M, K)

In [ ]:
plt.figure(figsize=(10,6), dpi=300)
plt.plot(K_values, K_estimates_MC, label='MC Arithmetic Estimate (M = 100.000)')
plt.plot(K_values, K_estimates_c, label='Control Variate Estimate (M = 100.000)')
# plt.plot(K_values, K_estimates_MC_1000, label='MC Arithmetic Estimate (M = 1.000)')
# plt.plot(K_values, K_estimates_c_1000, label='Control Variate Estimate (M = 1.000)')
plt.xlabel("Strike Price K (EUR)")
plt.ylabel("Call Option Price Approximation (EUR)")
plt.title("Performance of Control Variates For Increasing Strike Prices")
plt.legend()
plt.grid()
plt.show()

## Changing N (number of intervals) 

In [ ]:
S0 = 100
T = 1.
K = 99
r = 0.06
sigma = 0.2
M = 100_000

num_samples = 100
N_values = np.linspace(1, 1000, num=num_samples, dtype=int)
print(N_values)
    
N_estimates_c = np.zeros(num_samples)
N_estimates_MC = np.zeros(num_samples)
for i in tqdm(range(num_samples)):
    N = N_values[i]
    N_estimates_c[i], N_estimates_MC[i] = control_variate_est(S0, r, sigma, T, N, M, K)

In [ ]:
plt.figure(figsize=(10,6), dpi=300)
plt.plot(N_values, N_estimates_MC, label='MC Arithmetic Estimate')
plt.plot(N_values, N_estimates_c, label='Control Variate Estimate')
plt.xlabel("Number of intervals (N)")
plt.ylabel("Call Option Price Approximation (EUR)")
plt.title("Performance of Control Variates For Increasing Number of Intervals")
plt.legend()
plt.grid()
plt.show()

## Changing Sigma/Volatility

In [ ]:
S0 = 100
T = 1.
N = 50
K = 99
r = 0.06
M = 100_000

num_samples = 100
sigma_values = np.linspace(0.01, 10, num=num_samples)
print(sigma_values)

sigma_estimates_c = np.zeros(num_samples)
sigma_estimates_MC = np.zeros(num_samples)
for i in tqdm(range(num_samples)):
    sigma = sigma_values[i]
    sigma_estimates_c[i], sigma_estimates_MC[i] = control_variate_est(S0, r, sigma, T, N, M, K)

In [ ]:
plt.figure(figsize=(10,6), dpi=300)
plt.plot(sigma_values, sigma_estimates_MC, label='MC Arithmetic Estimate')
plt.plot(sigma_values, sigma_estimates_c, label='Control Variate Estimate')
plt.xlabel(r"Volatility ($\sigma$)")
plt.ylabel("Call Option Price Approximation (EUR)")
plt.title("Performance of Control Variates For Increasing Levels of Volatility")
plt.yscale('log')
plt.legend()
plt.grid()
plt.show()